<a href="https://colab.research.google.com/github/krsna540/Hackthons/blob/master/HRAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for heatmap plot
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [41]:
trainData = pd.read_csv("/content/train_LZdllcl.csv")
testData=pd.read_csv("/content/test_2umaH9m.csv")
print(trainData.shape)
print(testData.shape)

(54808, 14)
(23490, 13)


In [42]:
trainData.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [0]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

In [0]:
trainData['region'] = trainData['region'].map(lambda x: int(x.lstrip('region_')))
Gender = {'m': 1,'f': 0}
trainData.gender = [Gender[item] for item in trainData.gender]
trainData = pd.get_dummies(trainData, columns = ['recruitment_channel', 'department', 'education'], drop_first = True)

In [44]:
trainData.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [45]:
testData.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [0]:
testData['region'] = testData['region'].map(lambda x: int(x.lstrip('region_')))
Gender = {'m': 1,'f': 0}
testData.gender = [Gender[item] for item in testData.gender]
testData = pd.get_dummies(testData, columns = ['recruitment_channel', 'department', 'education'], drop_first = True)

In [47]:
testData.head()

,employee_id,region,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,recruitment_channel_referred,recruitment_channel_sourcing,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,education_Below Secondary,education_Master's & above
0,8724,26,1,1,24,NaN,1,1,0,77,0,1,0,0,0,0,0,0,0,1,0,0
1,74430,4,0,1,31,3.0,5,0,0,51,0,0,0,1,0,0,0,0,0,0,0,0
2,72255,13,1,1,31,1.0,4,0,0,47,0,0,0,0,0,0,0,0,1,0,0,0
3,38562,2,0,3,31,2.0,9,0,0,65,0,0,0,0,0,0,1,0,0,0,0,0
4,64486,29,1,1,30,4.0,7,0,0,61,0,1,1,0,0,0,0,0,0,0,0,0


In [0]:
train_x=trainData.drop(['is_promoted'],axis=1)
train_y=trainData['is_promoted']

In [49]:
train_x.head

<bound method NDFrame.head of        employee_id         department  ... awards_won? avg_training_score
0            65438  Sales & Marketing  ...           0                 49
1            65141         Operations  ...           0                 60
2             7513  Sales & Marketing  ...           0                 50
3             2542  Sales & Marketing  ...           0                 50
4            48945         Technology  ...           0                 73
5            58896          Analytics  ...           0                 85
6            20379         Operations  ...           0                 59
7            16290         Operations  ...           0                 63
8            73202          Analytics  ...           0                 83
9            28911  Sales & Marketing  ...           0                 54
10           29934         Technology  ...           0                 77
11           49017  Sales & Marketing  ...           0                 50
12      